In [1]:
import xml.etree.cElementTree as et
import pandas as pd
import os
import re
from tqdm import tqdm

## Notarieel archief

In [2]:
df_cols = ["uuid", "rubriek", "notaris", "inventarisNr", "akteNr", "akteType", 
           "datering", "taal", "beschrijving", "namen", 'urls']
df = pd.DataFrame(columns = df_cols)
for file in ["SAA_Index_op_notarieel_archief_20191105_001", 
             "SAA_Index_op_notarieel_archief_20191105_002",
             "SAA_Index_op_notarieel_archief_20191105_003",
             "SAA_Index_op_notarieel_archief_20191105_004",
             "SAA_Index_op_notarieel_archief_20191105_005",
             "SAA_Index_op_notarieel_archief_20191105_006",
             "SAA_Index_op_notarieel_archief_20191105_007",
             "SAA_Index_op_notarieel_archief_20191105_008",
             "SAA_Index_op_notarieel_archief_20191105_009",
             "SAA_Index_op_notarieel_archief_20191105_010"]:
    parsedXML = et.parse("notarieel/" + file + ".xml")
    root = parsedXML.getroot()
    rows = []

    for node in root: 
        if node is not None:
            s_uuid = node.find("uuid").text if node.find("uuid") is not None else None
            s_rubriek = node.find("rubriek").text if node.find("rubriek") is not None else None
            s_notaris = node.find("notaris").text if node.find("notaris") is not None else None
            s_inventarisNr = node.find("inventarisNr").text if node.find("inventarisNr") is not None else None
            s_akteNr= node.find("akteNr").text if node.find("akteNr") is not None else None
            s_akteType = node.find("akteType").text if node.find("akteType") is not None else None
            s_datering = node.find("datering").text if node.find("datering") is not None else None
            s_taal = node.find("taal").text if node.find("taal") is not None else None
            s_beschrijving = node.find("beschrijving").text if node.find("beschrijving") is not None else None
            s_namen = []
            if node.find("persoonsnamen") is not None:
                namen = []
                for x in node.find("persoonsnamen"):
                    voornaam = x.find("voornaam").text if x.find("voornaam") is not None else None
                    tussenvoegsel = x.find("tussenvoegsel").text if x.find("tussenvoegsel") is not None else None
                    achternaam = x.find("achternaam").text if x.find("achternaam") is not None else None
                    scanNaam = x.find("scanNaam").text if x.find("scanNaam") is not None else None
                    scanPositie = x.find("scanPositie").text if x.find("scanPositie") is not None else None
                    uuidNaam = x.find("uuidNaam").text if x.find("uuidNaam") is not None else None

                    namen.append({"voornaam": voornaam, "tussenvoegsel": tussenvoegsel, 
                         "achternaam": achternaam, "scanNaam": scanNaam, 
                         "scanPositie": scanPositie, "uuidNaam": uuidNaam})
            else:
                namen = None
            if node.find("urlScans") is not None:
                urls = []
                for x in node.find("urlScans"):
                    urlScan = x.text if x is not None else None
                    if urlScan != None:
                        urls.append(re.search(r'#.+', urlScan).group(0)[1:])
            else:
                urls = None
        else:
            s_uuid = None
            s_rubriek = None
            s_notaris = None
            s_inventarisNr = None
            s_akteNr= None
            s_akteType = None
            s_datering = None
            s_taal = None
            s_beschrijving = None
        rows.append({"uuid": s_uuid, "rubriek": s_rubriek, 
                     "notaris": s_notaris, "inventarisNr": s_inventarisNr, 
                     "akteNr": s_akteNr, "akteType": s_akteType,
                     "datering": s_datering, "taal": s_taal,
                     "beschrijving": s_beschrijving, "namen": namen, "urls":urls})

    out_df = pd.DataFrame(rows, columns = df_cols)
    df = df.append(out_df)
    df = df.reset_index(drop=True)


In [15]:
df

,uuid,rubriek,notaris,inventarisNr,akteNr,akteType,datering,taal,beschrijving,namen,urls
0,06306e00-98da-1130-ce03-93cbc225e77c,2,ELLERT DE VEER,6,160222,Testament,1587-03-18,nederlands,\nhandmerk\n,"[{'voornaam': 'Elisabeth', 'tussenvoegsel': 'd...",[KLAC00444000031.JPG]
1,122902ff-2633-2d1d-1fdd-0842fd195e3f,2,ELLERT DE VEER,6,162623,Insinuatie,1586-10-02,nederlands,None,"[{'voornaam': 'Aert', 'tussenvoegsel': None, '...",[KLAC00444000021.JPG]
2,22c4dd27-eec8-02bb-33d9-c8fea3161d18,2,ELLERT DE VEER,6,165855,Machtiging,1589-01-05,nederlands,None,"[{'voornaam': 'Feycke', 'tussenvoegsel': None,...",[KLAC00444000094.JPG]
3,2b017209-8640-4307-1423-df64615a59f6,2,ELLERT DE VEER,6,167432,Testament,1588-06-06,nederlands,\nCodicil. Legaat aan dienstmaagd.\n,"[{'voornaam': 'Pieter', 'tussenvoegsel': None,...","[KLAC00444000087.JPG, KLAC00444000088.JPG]"
4,30611e33-7c57-2b2a-bb0e-41227699255c,2,ELLERT DE VEER,6,168465,Machtiging,1587-06-23,nederlands,None,"[{'voornaam': 'Bartholomeus', 'tussenvoegsel':...","[KLAC00444000051.JPG, KLAC00444000052.JPG]"
...,...,...,...,...,...,...,...,...,...,...,...
292361,4d0a5598-5c6a-7040-e053-b784100a6264,507,Mr. AUGUST HENDRIK DE MELANDER,20034B,53690,Overig,1818-05-26,nederlands,None,"[{'voornaam': 'Johannes Daniel', 'tussenvoegse...","[KLAB02845000267.JPG, KLAB02845000268.JPG, KLA..."
292362,4d0a5598-6fb6-7040-e053-b784100a6264,507,Mr. AUGUST HENDRIK DE MELANDER,20034B,54446,Koop,1818-01-05,nederlands,None,"[{'voornaam': 'Hendrik', 'tussenvoegsel': None...","[KLAB02845000004.JPG, KLAB02845000005.JPG, KLA..."
292363,4d0a5598-8a27-7040-e053-b784100a6264,507,Mr. AUGUST HENDRIK DE MELANDER,20034B,55431,Testament,1818-06-23,nederlands,None,"[{'voornaam': 'Anke', 'tussenvoegsel': None, '...","[KLAB02845000313.JPG, KLAB02845000314.JPG]"
292364,4d0a5598-8a45-7040-e053-b784100a6264,507,Mr. AUGUST HENDRIK DE MELANDER,20034B,55436,Machtiging,1818-06-30,nederlands,None,"[{'voornaam': 'Trijntje', 'tussenvoegsel': Non...","[KLAB02845000318.JPG, KLAB02845000319.JPG]"


In [4]:
check = df[df.notaris.str.contains('JAN VERLEIJ')].reset_index(drop=True)

In [18]:
check.drop('rubriek', axis=1)

,uuid,notaris,inventarisNr,akteNr,akteType,datering,taal,beschrijving,namen,urls
0,17599c0c-3305-165c-aae3-eddbb497e4b4,JAN VERLEIJ,11885,107542,Testament,1741-03-16,nederlands,None,"[{'voornaam': 'Trijntje', 'tussenvoegsel': Non...","[KLAB05372000012.JPG, KLAB05372000013.JPG, KLA..."
1,1b037028-bea5-0fa8-e6c5-68155f5f2b21,JAN VERLEIJ,11885,108046,Attestatie,1741-07-26,nederlands,\nordentelijke vrijage\n,"[{'voornaam': 'Gerard', 'tussenvoegsel': None,...","[KLAB05372000193.JPG, KLAB05372000194.JPG, KLA..."
2,3d072c4e-8cf9-daf8-878d-a64133360fc8,JAN VERLEIJ,11885,112381,Insinuatie,1741-09-27,nederlands,\nschuld opgeeist\n,"[{'voornaam': 'Dirk', 'tussenvoegsel': 'van de...","[KLAB05372000281.JPG, KLAB05372000282.JPG, KLA..."
3,3ddb90b2-16e5-8f87-fda9-1bfb2804bb09,JAN VERLEIJ,11885,112488,Attestatie,1741-09-08,nederlands,\naantasting goede naam na overnachtingen na e...,"[{'voornaam': 'Sara', 'tussenvoegsel': None, '...","[KLAB05372000244.JPG, KLAB05372000245.JPG, KLA..."
4,412e645c-db5d-673e-5fce-006cfab5d7f1,JAN VERLEIJ,11885,112918,Testament,1741-12-29,nederlands,None,"[{'voornaam': 'Aaffje Jans', 'tussenvoegsel': ...","[KLAB05372000380.JPG, KLAB05372000381.JPG, KLA..."
...,...,...,...,...,...,...,...,...,...,...
13058,dcc03903-6c44-8ed3-2e3e-4539f500958c,JAN VERLEIJ,11967,99275,Kwitantie,1770-01-17,nederlands,\nKwitantie voor ontvangst van nalatenschap.\n,"[{'voornaam': 'Theodorus', 'tussenvoegsel': 'v...","[KLAB07041000258.JPG, KLAB07041000259.JPG]"
13059,e9390ff6-153c-2679-e855-53d67352ea9c,JAN VERLEIJ,11967,101127,Machtiging,1769-03-22,nederlands,\nVOC schip de Jonge Samuel. betaling uit gage...,"[{'voornaam': 'Pieter', 'tussenvoegsel': 'van'...","[KLAB07041000174.JPG, KLAB07041000175.JPG, KLA..."
13060,ec3d450b-486e-7e87-8894-adfd36d5f544,JAN VERLEIJ,11967,101568,Kwitantie,1770-08-11,nederlands,None,"[{'voornaam': 'Annitje', 'tussenvoegsel': None...","[KLAB07041000307.JPG, KLAB07041000308.JPG, KLA..."
13061,fe9f2abb-f67c-f214-23ec-41d724ed5ce7,JAN VERLEIJ,11967,104353,Scheepsverklaring,1769-02-22,nederlands,\nSchip des Jonge Fontijns heeft vertraging op...,"[{'voornaam': 'Jelle', 'tussenvoegsel': None, ...","[KLAB07041000100.JPG, KLAB07041000101.JPG, KLA..."


In [6]:
sum([len(x) for x in check.urls]) / len(check.urls)

3.2637219627956826

## Scanned Data

In [2]:
def namespace(element):
    m = re.match(r'\{.*\}', element.tag)
    return m.group(0) if m else ''

In [3]:
files = []
directory = 'D:\Documenten\Studie\Master\Scriptie\scanned\page'
for filename in os.listdir(directory):
    if filename.endswith(".xml"):
        files.append(filename)
    else:
        continue

In [4]:
df_cols = ["page", "textregion"]
scan_df = pd.DataFrame(columns = df_cols)
for file in files:
    parsedXML = et.parse("scanned/page/" + file)
    root = parsedXML.getroot()
    rows = []
    page = file[:-3] + "JPG"
    text = ""
    for node in root: 
        nspace = namespace(node)
        if node is not None:
            for child in node:
                if child.find(nspace + "TextEquiv") is not None:
                    regio = (child.find(nspace + "TextEquiv").find(nspace + "Unicode").text)
                    
    rows.append({"page": page, "textregion": regio })
    out_df = pd.DataFrame(rows, columns = df_cols)
    scan_df = scan_df.append(out_df)
    scan_df = scan_df.reset_index(drop=True)


In [10]:
scan_df.page.value_counts()

KLAB05727000080.JPG    27
KLAB05727000087.JPG    26
KLAB05727000082.JPG    24
KLAB05727000081.JPG    24
KLAB05727000084.JPG    23
                       ..
KLAB05650000157.JPG     1
KLAB05935000178.JPG     1
KLAB05747000230.JPG     1
KLAB05657000399.JPG     1
KLAB06775000409.JPG     1
Name: page, Length: 43184, dtype: int64

scan_df = scan_df.textregion.dropna()

In [14]:
scan_df.tail(10)

,page,textregion
96804,NOTG00445000218.JPG,Danit Durs hnoorman ƒ 10114:\nƒ 134: 195 : 209...
96805,NOTG00445000218.JPG,4\nhiodina Wesuwsets) Anoopmuast„ 20:\nJncoit ...
96806,NOTG00445000218.JPG,Jol\nHoudrick Boudin ab:\n12\nJant Mrcerus\nJo...
96807,NOTG00445000218.JPG,Voort 0o Jerde badstƒ 417104710:\n49 : 50 : 55...
96808,NOTG00445000219.JPG,Jan Janshur Brugse 21:\nJan Oinins Odibsse hou...
96809,NOTG00445000219.JPG,180 4
96810,NOTG00445000219.JPG,Als\nHKoete Janbuase\n5:10\nWloets Janoerns\n4...
96811,NOTG00445000219.JPG,\nWistse Bouwet huwgestse\n1 If0\nDouueseb Jan...
96812,NOTG00445000219.JPG,rijes Jar 15\n5\nDomast Soutse\n50 Jfe:\nLidum...
96813,NOTG00445000220.JPG,\nPisstass Moevenn 10: Las\ndweuwj darteus\n-\...


In [23]:
[[scan_df.textregion[x] for x in (range(len(scan_df))) if page== scan_df.page[x]] for page in tqdm(scan_df.page) if page not in complete_page]



  0%|                                                                                        | 0/96814 [00:00<?, ?it/s]

  0%|                                                                             | 16/96814 [00:01<3:05:24,  8.70it/s]

  0%|                                                                            | 17/96814 [00:03<16:29:43,  1.63it/s]

  0%|                                                                            | 18/96814 [00:05<25:42:15,  1.05it/s]

  0%|                                                                            | 19/96814 [00:07<32:26:01,  1.21s/it]

  0%|                                                                            | 20/96814 [00:08<37:14:49,  1.39s/it]

  0%|                                                                            | 21/96814 [00:10<40:16:12,  1.50s/it]

  0%|                                                                            | 22/96814 [00:12<41:58:07,  1.56s/it]

  0%|                         

KeyboardInterrupt: 